In [4]:
import numpy as np
import pandas as pd
import numpy.random as rng
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('iso.csv')
df = df[df['r1_charge_heater'] >= 0]

min_yield_perc = 99
min_ron = 83

valid_comb_ix = (df['process_ron'] >= min_ron) & (df['process_yield'] >= min_yield_perc)

# Random Experimentation

In [ ]:
def build_trace(df, chosen_ixs, min_ron, min_yield):
    
    traced_ch = df.iloc[chosen_ixs]['r1_charge_heater']
    traced_ron = df.iloc[chosen_ixs]['process_ron']
    traced_yield = df.iloc[chosen_ixs]['process_yield']
    
    feasibles_ix = (traced_ron >= min_ron) & (traced_yield >= min_yield)
    
    adjusted_ch = traced_ch * feasibles_ix + (1-feasibles_ix) * -1
    adjusted_ch[adjusted_ch==-1]=np.inf
    
    best_ch = nanmin_accumulate(adjusted_ch)
    return best_ch, feasibles_ix

def nanmin_accumulate(vals):
    acc = []
    min_val = float("inf")
    for val in vals:
        if val < min_val:
            min_val = val
        acc.append(min_val)
    return acc 
N_EXPS = 500
N_REPS = 100

traces = []
f_traces = []
for r in range(N_REPS):
    random_ix = rng.permutation(df.shape[0])[:N_EXPS]
    best_ch, feasibles_ix = build_trace(df, random_ix, min_ron, min_yield_perc)
    traces.append(best_ch)
    f_traces.append(feasibles_ix)

avg_best_random = np.mean(traces, axis=0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.plot(np.arange(avg_best_random.shape[0]), avg_best_random, linewidth=5, label='Random')
ax.plot(np.arange(avg_best_random.shape[0]), np.ones(avg_best_random.shape[0]) * 
        np.min(df[valid_comb_ix]['r1_charge_heater']), linewidth=5, linestyle='--', label='True Min')
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)
ax.set_xlabel('# Experiments', fontsize=28, fontweight='bold')
ax.set_ylabel('Best Feasible Charge Heater', fontsize=28, fontweight='bold')
ax.grid(True)
ax.legend(fontsize=28, frameon=False)

# Bayesian Optimization

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from scipy.stats import norm

In [ ]:
X = np.array(df[['r1_temp', 'r2_temp', 'r1_pressure', 'r2_pressure']])
y = np.array(df[['r1_charge_heater']])

In [ ]:
# surrogate or approximation for the objective function
def surrogate(model, X):
	# catch any warning generated when making a prediction
	with catch_warnings():
		# ignore generated warnings
		simplefilter("ignore")
		return model.predict(X, return_std=True)

# probability of improvement acquisition function
def acquisition(X, Xsamples, avail_indecies, model):
    
	# calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = min(yhat)
    
    #print(best)
	# calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)

    mu = mu[:, 0]
	# calculate the probability of improvement
    probs = norm.cdf((best - mu) / (std+1E-9))
    
    # don't pick anything that has already been used
    
    probs[~avail_indecies] = 0
    
    return probs

# optimize the acquisition function
def opt_acquisition(Xsamples, avail_indecies, X, y, model):
	
	# calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, avail_indecies, model)
    
    # locate the index of the largest scores
    ix = rng.permutation(scores.shape[0])
    best_ix = max(ix, key=lambda i: scores[i])
    
    return best_ix


In [ ]:
N_SAMPLES_TO_DRAW = 50
REPS = 100

best_chs = []
indecies = set(range(X.shape[0]))

for r in range(REPS):
    # initialize the model
    ix = rng.choice(X.shape[0], 1)
    Xtrain = X[ix,:]
    ytrain = y[ix,:] * valid_comb_ix[ix] + (1-valid_comb_ix[ix]) * 1e9
    
    model = GaussianProcessRegressor()
    model.fit(Xtrain, ytrain)
    
    best_ch = [ytrain[0,0]]
    # perform the optimization process
    avail_indecies = np.ones(X.shape[0]).astype(bool)
    avail_indecies[ix] = False 
   
    for i in range(N_SAMPLES_TO_DRAW-1):

        # select the next point to sample
        #ix = rng.permutation(X.shape[0])[:N_SAMPLES_TO_DRAW]
        #Xsamples = X[ix,:]
        ix = opt_acquisition(X, avail_indecies, Xtrain, ytrain, model)
        avail_indecies[ix] = False
        
        # sample the point
        actual = y[ix,:] * valid_comb_ix[ix] + (1-valid_comb_ix[ix]) * 1e9
        
        # add the data to the dataset
        Xtrain = np.vstack((Xtrain, X[ix,:]))
        ytrain = np.vstack((ytrain, actual))
        best_ch.append(np.min(ytrain))
        
        
        # update the model
        model.fit(Xtrain, ytrain)
    best_chs.append(best_ch)
best_chs = np.array(best_chs)

avg_best_bo = np.mean(best_chs, axis=0)
avg_best_bo

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.plot(np.arange(avg_best_bo.shape[0]), avg_best_bo, linewidth=5, label='BayesOpt')
ax.plot(np.arange(avg_best_random.shape[0]), avg_best_random, linewidth=5, label='Random')
ax.plot(np.arange(avg_best_random.shape[0]), np.ones_like(r1_charge_heater) * np.min(r1_charge_heater[valid_comb_ix]), linewidth=5, linestyle='--', label='True Min')
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)
ax.set_xlabel('# Experiments', fontsize=28, fontweight='bold')
ax.set_ylabel('Best Charge Heater', fontsize=28, fontweight='bold')
ax.set_xlim([0, N_SAMPLES_TO_DRAW])
ax.grid(True)
ax.set_yscale('log')
ax.legend(fontsize=28, frameon=False)

In [ ]:
np.min(r1_charge_heater[valid_comb_ix])

# Alternative Bayesian Optimization

In [ ]:
class SurrogateModel:
    
    def __init__(self):
        self._X = []
        self._y = []
        
    def add_observation(self, x, y):
        self._X.append(x)
        self._y.append(y)
        
        Xtrain, ytrain = self.get_train_data()
#         print("Xtrain shape: ", Xtrain.shape)
#         print("ytrain shape: ", ytrain.shape)
        self._model = GaussianProcessRegressor()
    
        self._model.fit(Xtrain, ytrain)
        
    def predict(self, X):
        # catch any warning generated when making a prediction
        with catch_warnings():
            # ignore generated warnings
            simplefilter("ignore")
            return self._model.predict(X, return_std=True)
    
    def prob_less_than(self, X, thres):
        mu, std = self.predict(X)

        mu = mu[:,0]
        
        probs = norm.cdf((thres - mu) / (std+1E-9))
        
        return probs 
    
    def prob_greater_than(self, X, thres):
        mu, std = self.predict(X)

        mu = mu[:,0]
        
        probs = norm.cdf((mu - thres) / (std+1E-9))
        
        return probs 
    
    def get_train_data(self):
        return np.array(self._X), np.array(self._y)

def plot_trace(mean_trace, col_names):
    f, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

    for c in range(len(col_names)):
        ax = axes[c]
        
        axes[c].plot(np.arange(mean_trace.shape[0]), mean_trace[:,c], linewidth=5, label=col_names[c])
    
        ax.tick_params(axis='x', labelsize=22)
        ax.tick_params(axis='y', labelsize=22)
        ax.set_xlabel('# Experiments', fontsize=28, fontweight='bold')
        ax.set_ylabel('Best', fontsize=28, fontweight='bold')
        ax.set_xlim([0, N_SAMPLES_TO_DRAW])
        ax.grid(True)
        ax.set_yscale('log')
        ax.legend(fontsize=28, frameon=False)

X = np.array(df[['r1_temp', 'r2_temp', 'r1_pressure', 'r2_pressure']])
ys_ch = np.array(df[['r1_charge_heater']])
ys_ron = np.array(df[['process_ron']])
ys_yield = np.array(df[['process_yield']])
min_ron = 83
min_yield = 99

N_SAMPLES_TO_DRAW = 50
REPS = 10

traces = []
f_traces = []
for r in range(REPS):
    
    # pick a random point to seed the model
    ix = rng.choice(X.shape[0])
    x = np.squeeze(X[ix,:])
    y_ch = ys_ch[ix]
    y_ron = ys_ron[ix]
    y_yield = ys_yield[ix]
    
    # instantiate three sms
    ch_sm = SurrogateModel()
    ch_sm.add_observation(x, y_ch)
    ron_sm = SurrogateModel()
    ron_sm.add_observation(x, y_ron)
    yield_sm = SurrogateModel()
    yield_sm.add_observation(x, y_yield)
    
    # keep track of available experiments to run
    avail_indecies = np.ones(X.shape[0]).astype(bool)
    avail_indecies[ix] = False 
    
    # track choices
    chosen_ixs = [ix]
    
    for i in range(N_SAMPLES_TO_DRAW-1):
        
        
        # calculate the probability of improving on the best charge heater value
        Xtrain, ytrain_ch = ch_sm.get_train_data()
        yhat_ch, _ = ch_sm.predict(Xtrain)
        best_ch = min(yhat)
         
        pi = ch_sm.prob_less_than(X, best_ch)
        
        # calculate the probability that the ron will be higher than given threshold
        p_ron_feasible = ron_sm.prob_greater_than(X, min_ron)
        
        # calculate the probability that the yield will be higher than given threshold
        p_yield_feasible = yield_sm.prob_greater_than(X, min_yield)
        
        # now maximize the product of three probs
        prob_goodness = p_ron_feasible * p_yield_feasible
        #print(np.max(prob_goodness))
        
        prob_goodness[~avail_indecies] = 0
        ix = rng.permutation(prob_goodness.shape[0])
        next_ix = max(ix, key=lambda i: prob_goodness[i])
        
        # add results
        x = np.squeeze(X[next_ix,:])
        y_ch = ys_ch[next_ix]
        y_ron = ys_ron[next_ix]
        y_yield = ys_yield[next_ix]
        ch_sm.add_observation(x, y_ch)
        ron_sm.add_observation(x, y_ron)
        yield_sm.add_observation(x, y_yield)
    
        # trace it
        avail_indecies[next_ix] = False
        chosen_ixs.append(next_ix)
     
    best_ch, feasibles_ix = build_trace(df, chosen_ixs, min_ron, min_yield_perc)
    traces.append(best_ch)
    f_traces.append(feasibles_ix)
    
avg_best_bo = np.mean(f_traces, axis=0)
avg_best_bo

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.plot(np.arange(avg_best_random.shape[0]), avg_best_random, linewidth=5, label='Random')
ax.plot(np.arange(avg_best_bo.shape[0]), avg_best_bo, linewidth=5, label='Bayesopt')
ax.plot(np.arange(avg_best_random.shape[0]), np.ones(avg_best_random.shape[0]) * 
        np.min(df[valid_comb_ix]['r1_charge_heater']), linewidth=5, linestyle='--', label='True Min')
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)
ax.set_xlabel('# Experiments', fontsize=28, fontweight='bold')
ax.set_ylabel('Best Feasible Charge Heater', fontsize=28, fontweight='bold')
ax.grid(True)
ax.legend(fontsize=28, frameon=False)

# Stochastic NN

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import scipy.stats as stats
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')


Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:

class StochasticNN:
    
    def __init__(self, cfg):
        self._cfg = cfg 
        
        
    def train(self, Xtrain, Ytrain):
        
        self._make_model()
        
        # normalize X and y train
        Xtrain = stats.zscore(Xtrain, axis=0, ddof=1)
        Ytrain_mu = np.mean(Ytrain, axis=0, keepdims=True)
        Ytrain_std = np.std(Ytrain, axis=0, ddof=1, keepdims=True)
        Ytrain = (Ytrain - Ytrain_mu) / Ytrain_std
        
        self._Ytrain_mu = Ytrain_mu
        self._Ytrain_std = Ytrain_std
        
        history = self._model.fit(Xtrain, 
                                  Ytrain, 
                                  batch_size=Xtrain.shape[0]//10, 
                                  epochs=self._cfg['n_epochs'], 
                                  verbose=self._cfg['verbose'])
        
    def _make_model(self):
        cfg = self._cfg 
        
        keras.backend.clear_session()
    
        input_layer = keras.layers.Input(shape=(cfg['n_input'],))
        hidden_layer = keras.layers.Dense(cfg['n_hidden'], activation='tanh')(input_layer)
        dropout1_layer = keras.layers.Dropout(0.5)(hidden_layer, training=True)
        output_layer = keras.layers.Dense(cfg['n_output'], activation='linear')(dropout1_layer)
        
        model = keras.Model(input_layer, output_layer)
        model.compile(
            loss=keras.losses.MeanAbsoluteError(),
            optimizer=keras.optimizers.Nadam()
        )
        #print(model.summary())
        self._model = model 
    
    def predict(self, X, n_samples=2):
        
        X = stats.zscore(X, axis=0, ddof=1)
        
        samples = []
        for i in range(n_samples):
            preds = self._model.predict(X) * self._Ytrain_std + self._Ytrain_mu 
            samples.append(preds)
        
        samples = np.array(samples)
        return samples

In [6]:

X = np.array(df[['r1_temp', 'r2_temp', 'r1_pressure', 'r2_pressure']])
Y = np.array(df[['r1_charge_heater', 'process_ron', 'process_yield']])

min_ron = 83
min_yield = 99

N_INIT_SAMPLES = 10
N_SAMPLES_TO_DRAW = 50
N_PRED_SAMPLES=20
REPS = 10

nn = StochasticNN({ "n_input" : 4, "n_output" : 3, "n_hidden" : 5, "n_epochs" : 50, "verbose" : False })

bo_traces = []
for r in range(REPS):
    
    # pick random points to seed the model
    chosen_ix = rng.choice(X.shape[0], N_INIT_SAMPLES, replace=False).tolist()
    
    # keep track of available experiments to run
    avail_indecies = np.ones(X.shape[0]).astype(bool)
    avail_indecies[chosen_ix] = False 
    
    while len(chosen_ix) < N_SAMPLES_TO_DRAW:
        #print("%r %d " % (r, len(chosen_ix)))
        # train model
        Xtrain = X[chosen_ix, :]
        Ytrain = Y[chosen_ix, :]
        print("Latest obs: %s" % Ytrain[-1,:])
        nn.train(Xtrain, Ytrain)
        
        # sample predictions
        pred_samples = nn.predict(X, n_samples=N_PRED_SAMPLES)
        
        # compute the probability of feasibility
        prob_goodness = np.mean((pred_samples[:,:,1] >= min_ron) & (pred_samples[:,:,2] >= min_yield), axis=0)
        prob_goodness[chosen_ix] = 0
        #print("Max prob goddness: %0.2f" % np.max(prob_goodness))
        # choose next point
        ix = rng.permutation(prob_goodness.shape[0])
        next_ix = max(ix, key=lambda i: prob_goodness[i])
        chosen_ix.append(next_ix)
    
    bo_traces.append(chosen_ix)

Latest obs: [4.50380974e+06 8.35797172e+01 9.79699929e+01]
Metal device set to: Apple M1


2021-08-06 11:52:39.781814: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-06 11:52:39.781984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-08-06 11:52:39.839934: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-06 11:52:39.840127: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-06 11:52:39.968294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-06 11:52:42.676014: I tensorflow/core/grappler/

Latest obs: [3.07846301e+06 8.02640979e+01 1.00258603e+02]


2021-08-06 11:52:48.249573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-06 11:52:51.227793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [ ]:
bo_ch_traces = []
bo_f_traces = []
for chosen_ix in bo_traces:
    print(chosen_ix)
    best_ch, feasibles_ix = build_trace(df, chosen_ix, min_ron, min_yield)
    bo_ch_traces.append(best_ch)
    bo_f_traces.append(feasibles_ix)

avg_best_bo = np.mean(bo_ch_traces, axis=0)
np.mean(bo_f_traces, axis=0)